In [ ]:
import io
from PIL import Image
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import json
import torch.utils.data as tdata
from tqdm import tqdm
from torch import optim
import torch.nn as nn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from transformers.models.detr.feature_extraction_detr import rgb_to_id
from transformers import AutoImageProcessor, AutoModelForImageClassification

In [ ]:
seed_value = 7
torch.manual_seed(seed_value)
if torch.cuda.is_available():
  torch.cuda.manual_seed(seed_value)
  torch.cuda.manual_seed_all(seed_value)
np.random.seed(seed_value)

In [ ]:
device= torch.device("cuda:0")
device

In [ ]:
# %pip install --upgrade pyarrow
# %pip install -qq -U diffusers datasets transformers accelerate ftfy pyarrow

In [ ]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("keremberke/excavator-detector", "full")

In [ ]:

model.roi_heads.box_predictor = FastRCNNPredictor(model.roi_heads.box_predictor.cls_score.in_features, 3)

In [ ]:
class Dataset(tdata.Dataset):
    def __init__(self,dataset):

        self.image=[]
        self.type=[]
        self.bbox=[]
        for id in range(len(dataset['image'])):
            self.image.append(dataset['image'][id])
            self.type.append(dataset['objects'][id]['category'][0]-1)
            box=np.array(dataset['objects'][id]['bbox'][0])
            self.bbox.append(box)
                
        self.len=len(self.image)
    
    def __getitem__(self, id):
        return  torch.from_numpy(np.array(self.image[id])).type(dtype=torch.float),torch.from_numpy(self.bbox[id]).type(dtype=torch.float)

    def __len__(self):
        return self.len
    


In [ ]:
train_set=Dataset(dataset['train'][0:104])

In [ ]:
train_set[0]

In [111]:
images, boxes = torch.rand(1, 3, 600, 1200), torch.tensor([0,0,1,1])
v=[{'labels':torch.tensor([[1]],dtype=torch.int64).to(device),'boxes':torch.tensor([[0,0,1,1]],dtype=torch.float32)}]

#different outputs
model.train()
out=model(images,[{'labels':torch.tensor([1],dtype=torch.int64),'boxes':torch.tensor([[0,0,1,1]],dtype=torch.float32)}])
i=0

model.eval()
out=model(images,[{'labels':torch.tensor([1],dtype=torch.int64),'boxes':torch.tensor([[0,0,1,1]],dtype=torch.float32)}])
i=0

In [108]:

torch.flip(train_set[0][0].unsqueeze(0), dims=[0,1]).shape

torch.Size([1, 640, 640, 3])

In [ ]:
train_set[0][1]